# Ansible Galaxy Redfish collections

Version 0.112

## Introduction

The Ansible [Galaxy community.general collection](https://galaxy.ansible.com/community/general) proposes three Redfish modules:
  
  * [Redfish info](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_info_module.html) - Retrieves information like Systems inventory and Accounts inventory. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_info.py). 
  * [Redfish command](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_command_module.html) - Performs `set` operations like log management, user management, and power operations (e.g. on, off, reboot, etc.) on the remote BMC. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_command.py) 
  * [Redfish config](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_config_module.html) - Performs operations like setting the BIOS configuration. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_config.py) 
  
The above Ansible Redfish modules, are based upon the [redfish_utils.py](https://github.com/ansible-collections/community.general/blob/main/plugins/module_utils/redfish_utils.py) python utility module.


Redfish Ansible playbooks using the above modules are present in the [DMTF Redfish-Ansible-Playbooks](https://github.com/DMTF/Redfish-Ansible-Playbooks) public GitHub repository.

> **Note**: Redfish OEM extension operations using the Ansible Galaxy modules can be performed using the [DMTF](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md) instructions


## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.12) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=4
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="4/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/4/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "Good News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port={{ ILOSYBASEPORT }}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
username="student"
password="password"
__EOF__


## TBD



In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null

# Populate Python Venv with the HPE python-ilorest-library
pip install wheel                                               &>/dev/null
pip install python-ilorest-library                              &>/dev/null

# Install latest Ansible in the Venv
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

## redfish_info example

In [ ]:
ansible-playbook -i ${InvFile} ${NbId}/GetChassisInventory.yml

## Redfish_command example

In [ ]:
# Set Indicator LED
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLED.yml

## Redfish info example: category Manager

In [ ]:
# Get Virtual Media information
ansible-playbook -i ${InvFile} ${NbId}/GetVirtualMediaInfo.yml

## Insert Virtual Media

In [ ]:
ansible-playbook -i ${InvFile} ${NbId}/InsertVirtualMedia.yml

In [ ]:
ResetSimulators